In [1]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB as MNB
from sklearn.pipeline import Pipeline

In [2]:
import re
import string
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

In [3]:
tweet_df_location = pd.read_csv('../data/graphAnalysis/clean_climateTwitterData.csv')
tweet_df_location = tweet_df_location[tweet_df_location['text'].notna()]

In [4]:
# This step was performed in the sentiment Analyzer
# def replace_urls(in_string, replacement=None):
#     # """Replace URLs in strings. See also: ``bit.ly/PyURLre``

#     # Args:
#     #     in_string (str): string to filter
#     #     replacement (str or None): replacment text. defaults to '<-URL->'

#     # Returns:
#     #     str
#     # """
#     replacement = '<-URL->' if replacement is None else replacement
#     pattern = re.compile('(https?://)?(\w*[.]\w+)+([/?=&]+\w+)*')
#     return re.sub(pattern, replacement, in_string)


def tokenize_only(in_string):
    """
    Convert `in_string` of text to a list of tokens using NLTK's TweetTokenizer
    """
    # reasonable, but adjustable tokenizer settings
    tokenizer = TweetTokenizer(preserve_case=False,
                               reduce_len=True,
                               strip_handles=False)
    tokens = tokenizer.tokenize(in_string)
    return tokens




In [5]:
# This step was performed in the sentiment Analyzer
# # remove urls and retweets and entities from the text
# tweet_df_location['text_clean'] = tweet_df_location['text'].apply(lambda row:replace_urls(row))

# #remove punctuations
# RE_PUNCTUATION = '|'.join([re.escape(x) for x in string.punctuation])  
# tweet_df_location['text_clean'] = tweet_df_location['text_clean'].str.replace(RE_PUNCTUATION, "")

In [6]:


# List of stopwords
stop_words= stopwords.words('english') #import stopwords from NLTK package
readInStopwords = pd.read_csv("pre_process/twitterStopWords.csv", encoding='ISO-8859-1') # import stopwords from CSV file as pandas data frame
readInStopwords = readInStopwords.wordList.tolist() # convert pandas data frame to a list
readInStopwords.append('http')
readInStopwords.append('https')

search_terms = ['#climateStrike','#climatestrike','#climatechange','#GreenNewDeal','#climatecrisis','#climateAction','#FridaysForFuture',
            '#environment','#globalwarming','#GlobalWarming','#ActOnClimate','#sustainability','#savetheplanet',
        '#bushfiresAustralia','#bushfires']
readInStopwords.extend(search_terms)
stop_list = stop_words + readInStopwords # combine two lists i.e. NLTK stop words and CSV stopwords
stop_list = list(set(stop_list)) # strore only unique values 
    
print(stop_list)

["'tis", 'whom', ' she', 'how', ' if', "won't", "she'll", 'own', 'we', 'your', 'yours', 'in', " must've", ' ever', 'nor', ' but', " when'd", "i'd", 'should', ' so', ' only', 'those', 'after', 'isn', ' into', 'there', ' every', ' then', 'about', " there's", 'such', '#GreenNewDeal', 'don', ' either', ' were', "you'll", " couldn't", ' will', " doesn't", ' got', " they've", ' at', '#savetheplanet', ' must', 'between', " when's", "why's", "didn't", 'theirs', 'further', " how'll", 'yourself', 'up', 'doesn', ' them', ' who', "they're", ' let', 'http', "shan't", 'or', " i'll", ' twas', 'd', 'then', "hasn't", ' just', 'shan', 'ours', "aren't", "where's", 'wasn', ' in', 'for', "i'm", "let's", 'could', " you'll", 'am', ' their', 'all', 'other', " why'd", "how's", 'y', "he's", 'were', ' we', 'doing', 'off', 'and', " i'm", ' said', 'have', " who'll", 'couldn', 'been', " you'd", 'weren', ' some', "doesn't", 'but', 'each', 'both', ' hers', 'ma', 'our', "when's", " shan't", ' dear', ' any', ' had', " 

In [7]:
X = tweet_df_location['text_clean']
y = tweet_df_location['search_hashtags']


In [8]:
# # Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)
encoded_y.shape
encoded_y

array([ 5,  5,  5, ..., 10, 10, 10])

In [9]:

tf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=0.00, stop_words=stop_list, tokenizer=tokenize_only) # Use tf (raw term count) features for LDA.
#X_Vect = tf_vectorizer.fit_transform(X)



In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, encoded_y, test_size=0.30)#, random_state=42, stratify=y)

In [11]:
X_train.shape

(50683,)

In [12]:
y_train.shape

(50683,)

In [13]:
X_test.shape

(21722,)

In [14]:
y_test.shape

(21722,)

In [15]:
text_clf_pipe = Pipeline([
    ('tfidf',TfidfVectorizer(max_df=0.9, min_df=0.00, stop_words=stop_list, tokenizer=tokenize_only)),
    ('clf',MNB(alpha=0.01))])

In [16]:
text_clf_pipe.fit(X_train, y_train)  

C:\Users\tasne\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['#actonclimate', '#bushfiresaustralia', '#climateaction', '#fridaysforfuture', '#greennewdeal', "'", 'able', 'across', "ain't", 'almost', 'also', 'among', "can't", 'cannot', "could've", 'dear', 'either', 'else', 'ever', 'every', 'get', 'got', "how'd", "how'll", 'however', 'least', 'let', 'like', 'likely', 'may', 'might', "might've", 'must', "must've", 'neither', 'often', 'rather', 'said', 'say', 'says', 'since', 'tis', 'twas', 'us', 'wants', "what'd", "when'd", "when'll", "where'd", "where'll", "who'd", "who'll", "why'd", "why'll", "would've", 'yet'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.9, max_features=None,
                                 min_df=0.0, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=["'tis", 'whom', ' she', 'how',
                                             ' if', "won't", "she'll", 'own',
                                             '...
                                             " must've", ' ever', 'nor', ' but',
                                             " when'd", "i'd", 'should', ' so',
                                             ' only', 'those', 'after', 'isn',
                               

In [17]:
predictions = text_clf_pipe.predict(X_test)

In [18]:
print(confusion_matrix(y_test,predictions))

[[ 282    0   17  122   19   59   10    3    3   24    0   10]
 [   0  142    2  106   19    6    1    1    1    0    0    0]
 [  22    4  712  502  105  448   36   53   23   27    5   35]
 [  37   51  302 3177  200  505  146   41  143  100   23   85]
 [  22   15   77  523  511  209   26   14   37   56    4   20]
 [  13    2  109  354   91 4705   24  183   22   41    6   12]
 [   3    5   48  349   24   60  684    6   26   11   22  135]
 [   1    2   37   76   24  454   23  251    6    5   10   12]
 [   0    3    8  235   26   53   57   14  845    4   12    5]
 [  11    0   30  257   51  120    5    2    7  883    2   12]
 [   1    0   13   84    5   37   65    6   18    2  157   19]
 [   5    0   46  308   10   37  113    5   28    9   16 1137]]


In [19]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.71      0.51      0.60       549
           1       0.63      0.51      0.57       278
           2       0.51      0.36      0.42      1972
           3       0.52      0.66      0.58      4810
           4       0.47      0.34      0.39      1514
           5       0.70      0.85      0.77      5562
           6       0.57      0.50      0.53      1373
           7       0.43      0.28      0.34       901
           8       0.73      0.67      0.70      1262
           9       0.76      0.64      0.69      1380
          10       0.61      0.39      0.47       407
          11       0.77      0.66      0.71      1714

    accuracy                           0.62     21722
   macro avg       0.62      0.53      0.56     21722
weighted avg       0.62      0.62      0.61     21722



              precision    recall  f1-score   support

           0       0.71      0.51      0.60       549
           1       0.63      0.51      0.57       278
           2       0.51      0.36      0.42      1972
           3       0.52      0.66      0.58      4810
           4       0.47      0.34      0.39      1514
           5       0.70      0.85      0.77      5562
           6       0.57      0.50      0.53      1373
           7       0.43      0.28      0.34       901
           8       0.73      0.67      0.70      1262
           9       0.76      0.64      0.69      1380
          10       0.61      0.39      0.47       407
          11       0.77      0.66      0.71      1714

    accuracy                           0.62     21722
    macro avg      0.62      0.53      0.56     21722
    weighted avg   0.62      0.62      0.61     21722


In [20]:
metrics.accuracy_score(y_test,predictions)

0.6208452260381181

- Accuracy is -- 0.4743117576650401 without Tokenization and Stopwords
- Accuracy is -- 0.4820458521314796 with Tokenization and Stopwords

In [21]:
print('MultinomialNB Train: %.3f' % text_clf_pipe.score(X_train, y_train))
print('MultinomialNB Test Acc: %.3f' % text_clf_pipe.score(X_test, y_test))

MultinomialNB Train: 0.839
MultinomialNB Test Acc: 0.621


In [22]:
strVal = ["Cuyahoga county eliminates use of plastic bags","Weather is still warm in winters and is not freezing"]
# Value = tf_vectorizer.transform(strVal)
# Value.shape

In [28]:
#Use this way to obtain the value from encoding
inv_s = label_encoder.inverse_transform(text_clf_pipe.predict(strVal).astype(int).ravel())
inv_s

array(['#environment', '#globalwarming'], dtype=object)

In [25]:
from sklearn.model_selection import GridSearchCV
parameters_svm = {'tfidf__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False),'clf__alpha': (1e-2, 1e-3)}
gs_clf = GridSearchCV(text_clf_pipe, parameters_svm, n_jobs=-1)
gs_clf = gs_clf.fit(X_train, y_train)
print("Best cross-validation score: {:.2f}".format(gs_clf.best_score_))
print("Best parameters: ", gs_clf.best_params_)
# param_grid = {'alpha': [0.001, 0.01, 0.1, 1]}

# grid = GridSearchCV(MNB(), param_grid, cv=10)
# grid.fit(X_train, y_train)
# print("Best cross-validation score: {:.2f}".format(grid.best_score_))
# print("Best parameters: ", grid.best_params_)

C:\Users\tasne\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\tasne\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['#actonclimate', '#bushfiresaustralia', '#climateaction', '#fridaysforfuture', '#greennewdeal', "'", 'able', 'across', "ain't", 'almost', 'also', 'among', "can't", 'cannot', "could've", 'dear', 'either', 'else', 'ever', 'every', 'get', 'got', "how'd", "how'll", 'however', 'least', 'let', 'like', 'likely', 'may', 'might', "might've", 'must', "must've", 'neither', 'often', 'rather', 'said', 'say', 'says', 'since', 'tis', 'twas', 'us', 'wants', "what'd", "when'd", "when'll", "where'd", "where'll", "who'd", "who'll", "why'd", "why'll", "would'v

Best cross-validation score: 0.66
Best parameters:  {'clf__alpha': 0.01, 'tfidf__ngram_range': (1, 2), 'tfidf__use_idf': False}


Best cross-validation score: 0.67
Best parameters:  {'clf__alpha': 0.01, 'tfidf__ngram_range': (1, 2), 'tfidf__use_idf': False}

In [30]:
gs_clf.predict(strVal)

array([6, 8])

In [32]:
#Use this way to obtain the value from encoding
inv_s = label_encoder.inverse_transform(gs_clf.predict(strVal).astype(int).ravel())
inv_s

array(['#environment', '#globalwarming'], dtype=object)